In [11]:
# include libraries 
%matplotlib inline 

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import re

In [31]:
results = []

# create list with only .csv files 
files = os.listdir (".")
for f in files:
    if (re.search(r"csv", f)):
        results.append(f)

['heapsort.csv', 'selectionsort.csv', 'treesort.csv']


In [32]:
sdf = pd.read_csv(results[0], sep=',')

In [33]:
sdf.head()

,2,0.000001
0,4,0.000001
1,8,0.000002
2,16,0.000004
3,32,0.000010
4,64,0.000025
